In [28]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import resample

In [22]:
df = pd.read_csv("Default.csv")
df["default"] = df["default"].astype("category")
df["student"] = df["student"].astype("category")

In [23]:
y = (df["default"] == "Yes").values
df["student_dummy"] = (df["student"] == "Yes").astype(int)
columnas_posibles = ["balance", "income", "student_dummy"]
x = df[columnas_posibles]
columnas_posibles = x.columns

## Accuracy LogisticRegresion

In [24]:
X = df[columnas_posibles]

In [25]:
lr = LogisticRegression()
lr = make_pipeline(StandardScaler(), lr)
lr.fit(X, y)
y_pred = lr.predict(X)
accuracy = np.mean(y_pred == y)
print("Accuracy del modelo:", accuracy)

Accuracy del modelo: 0.9733


#Punto extra

In [26]:
from sklearn.metrics import roc_auc_score
y_pred = lr.predict_proba(X)[:, 1]
auc_score= roc_auc_score(y, y_pred)
print("AUC del modelo:", auc_score)

AUC del modelo: 0.9495553275422935


#Accuracy de Bagging

In [38]:
B = 5000
n_boot = 5000

all_preds = []

for b in range(B):

    cols_elegidas = np.random.choice(columnas_posibles, size=2, replace=False)

    X_full = df[cols_elegidas].values

    X_b, y_b = resample(
        X_full,
        y,
        replace=True,
        n_samples=n_boot
    )

    tree = DecisionTreeClassifier()
    tree.fit(X_b, y_b)

    y_pred_full = tree.predict(X_full)

    all_preds.append(y_pred_full)

preds_matrix = np.vstack(all_preds)

y_pred_bagging = (preds_matrix.mean(axis=0) >= 0.5).astype(int)

accuracy_bagging = np.mean(y_pred_bagging == y)

print("Accuracy del modelo bagging con árboles y bootstrap:",
      accuracy_bagging)

Accuracy del modelo bagging con árboles y bootstrap: 0.9846


#Comparación
En el modelo de regresión logística simple me da un accuracy de 0.9733, mientras que el bagging con árboles y bootstrap me da un accuracy de 0.9846. El modelo de árboles de decisión obtiene un accuracy superior, esto puede ser ya que los árboles capturan relaciones no lineales entre variables, aparte de que con el bagging se reduce la varianza del modelo, con un solo árbol se sobreajusta, pero con bootstrap y bagging cada árbol comete errores diferentes y así el modelo final es mucho más estable.

#Conclusión
El modelo de bagging con árboles de decisión se mostro con un accuracy de 0.9846 más alto a la regresión logística que es 0.9733.

Esta mejora se debe a que el bagging combina múltiples árboles sobre muestras de bootstrap del conjunto original, con lo cual se reduce la varianza. Además, con la moda permite una predicción final más estable y por ello se produce un predictor más preciso.